In [ ]:
# Small ESP8266

#      RST          |  GPIO1   TX
#       A0          |  GPIO3   RX
#       D0  GPIO16  |  GPIO5   D1  SCL
# SCK   D5  GPIO14  |  GPIO4   D2  SDA
# MISO  D6  GPIO12  |  GPIO0   D3
# MOSI  D7  GPIO13  |  GPIO2   D4  LED
# SS    D8  GPIO15  |          GND
#      3V3          |          5V

# (not able to get the mDNS to work for esp8266)

In [17]:
%serialconnect

serial exception on close write failed: [Errno 5] Input/output error
Connecting to --port=/dev/ttyUSB1 --baud=115200 
Ready.


In [6]:
%sendtofile config.txt

wifiname     DoESLiverpool
wifipassword decafbad00
mqttbroker   10.0.100.1
pinled       2
pinpir       13
boardname    esp8266pir1
wifialt      BV6000,bubblino,192.168.43.1

Sent 7 lines (172 bytes) to config.txt.


In [18]:
%sendtofile --quiet --source utils.py
%sendtofile --quiet --source stdmqttas.py

Sent 32 lines (969 bytes) to utils.py.
Sent 49 lines (1662 bytes) to stdmqttas.py.


In [8]:
%serialconnect

serial exception on close write failed: [Errno 5] Input/output error
Connecting to --port=/dev/ttyUSB0 --baud=115200 
Ready.


In [9]:
#%sendtofile main.py

import time
from machine import Pin
import uasyncio as asyncio
from stdmqttas import fconfig, config, mqttconnecttask, callbackcmdtask
from mqtt_as import MQTTClient

pinled = Pin(int(fconfig["pinled"]), Pin.OUT)
pinpir = Pin(int(fconfig["pinpir"]), Pin.IN)
boardname = fconfig["boardname"]    
topicstatus = boardname+"/status"
topicpir = boardname+"/pir"
topiccmd = boardname+"/cmd"
topicreply = boardname+"/reply"

def modloop(N):
    while True:
        yield from range(N)

pirstate = -1
async def pirchangedetectiontask(client):
    global pirstate
    for n in modloop(400):
        prevpirstate, pirstate = pirstate, pinpir.value()
        if pirstate != prevpirstate:
            if client.isconnected():
                await client.publish(topicpir, str(pirstate), retain=True)
            print("pir", pirstate)
        blinkstate = (n == 0) or ((not client.isconnected()) and 20<=n<=60)
        pinled.value((1-pirstate) != blinkstate)
        await asyncio.sleep_ms(5)

async def onconnecttask(client):
    global pirstate
    ipnumber = client._sta_if.ifconfig()[0]
    await client.publish(topicstatus, ipnumber, retain=True)
    await client.subscribe(topiccmd, 1)
    pirstate = -1
    
def callbackcmd(topic, msg, retained):
    aloop.create_task(callbackcmdtask(client, topicreply, msg))

config['subs_cb'] = callbackcmd
config['connect_coro'] = onconnecttask
config['will'] = (topicstatus, "offline", True)
client = MQTTClient(config)
    
aloop = asyncio.get_event_loop()
aloop.create_task(mqttconnecttask(client))
aloop.create_task(pirchangedetectiontask(client))
aloop.run_forever()



ip:10.0.36.2,mask:255.0.0.0,gw:10.0.0.1
sleep disable
await connecting to: DoESLiverpool 10.0.100.1
*** connected:
pir -1
pir 1
.
**[ys] <class 'serial.serialutil.SerialException'>
**[ys] read failed: device reports readiness to read but returned no data (device disconnected or multiple access on port?)


**[ys] <class 'serial.serialutil.SerialException'>
**[ys] read failed: device reports readiness to read but returned no data (device disconnected or multiple access on port?)



In [1]:
%serialconnect

Connecting to --port=/dev/ttyUSB0 --baud=115200 

**[ys] <class 'serial.serialutil.SerialException'>
**[ys] read failed: device reports readiness to read but returned no data (device disconnected or multiple access on port?)

[Timed out waiting for recognizable response]
Disconnecting [paste mode not working]
  (You may need to reset the device)

In [ ]:
"{:02X}{:02X}{:02X}".format(*network.WLAN().config('mac')[-3:])